## Morphometic processing

In [1]:
import geopandas as gpd
import pandas as pd
import glob

def get_processed_region_ids(directory):
    return [int(s.split('_')[-1].split('.')[0]) for s in glob.glob(directory + '*')]

In [2]:
# !mkdir /data/uscuni-eurofab-overture/processed_data/
# !mkdir /data/uscuni-eurofab-overture/processed_data/simplified_buildings/
# !mkdir /data/uscuni-eurofab-overture/processed_data/streets/
# !mkdir /data/uscuni-eurofab-overture/processed_data/enclosures/
# !mkdir /data/uscuni-eurofab-overture/processed_data/tessellations/
# !mkdir /data/uscuni-eurofab-overture/processed_data/neigh_graphs/
# !mkdir /data/uscuni-eurofab-overture/processed_data/chars/


In [3]:
regions_buildings_dir = '/data/uscuni-eurofab-overture/regions/buildings/'
buildings_dir = '/data/uscuni-eurofab-overture/processed_data/buildings/'
overture_streets_dir = '/data/uscuni-eurofab-overture/overture_streets/'
streets_dir = '/data/uscuni-eurofab-overture/processed_data/streets/'
enclosures_dir = '/data/uscuni-eurofab-overture/processed_data/enclosures/'
tessellations_dir = '/data/uscuni-eurofab-overture/processed_data/tessellations/'
graph_dir = '/data/uscuni-eurofab-overture/processed_data/neigh_graphs/'
chars_dir = '/data/uscuni-eurofab-overture/processed_data/chars/'
simplfied_buildings_dir = '/data/uscuni-eurofab-overture/processed_data/simplified_buildings/'


regions_datadir = "/data/uscuni-eurofab-overture/"

In [4]:
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "ov_ce_region_hulls.parquet"
    )
region_hulls.shape

(709, 1)

In [5]:
region_id = 53490

## Process buildings

Run the building processing pipeline for all regions

In [7]:
from core.generate_buildings import read_region_buildings, process_region_buildings

In [8]:
def process_single_region_buildings(region_id):
    print('processing', region_id)
    buildings = gpd.read_parquet(regions_buildings_dir + f'buildings_{region_id}.pq')
    buildings = process_region_buildings(buildings, True, simplification_tolerance=.1, merge_limit=25)
    buildings.to_parquet(simplfied_buildings_dir + f"buildings_{region_id}.parquet")

In [ ]:
%%capture cap
for region_id, _ in region_hulls.iterrows():
    process_single_region_buildings(region_id)

Copy over the simplified buildings to the processed data folder.

In [15]:
# !cp -r /data/uscuni-eurofab-overture/processed_data/simplified_buildings/ /data/uscuni-eurofab-overture/processed_data/buildings/

## Process streets

Run the street processing pipeline for all regions.

In [5]:
from core.generate_streets import process_region_streets

In [ ]:
%%capture cap
for region_id, _ in region_hulls.iterrows():
    streets = process_region_streets(region_id, overture_streets_dir, buildings_dir)
    streets.to_parquet(streets_dir + f'streets_{region_id}.parquet')

In [ ]:
region_id  = 109005

## Process elements

Run the element generating pipeline for all regions.

In [5]:
from core.generate_elements import process_region_elements, generate_enclosures_representative_points, generate_tess

In [6]:
processed_region_ids = get_processed_region_ids(tessellations_dir)


In [8]:
for region_id, _ in region_hulls[~region_hulls.index.isin(processed_region_ids)].iterrows():
    enclosures, tesselations = process_region_elements(buildings_dir, streets_dir, region_id)

    enclosures.to_parquet(enclosures_dir + f"enclosure_{region_id}.parquet")
    print("Processed enclosures")
    
    ## save files
    tesselations.to_parquet(
        tessellations_dir + f"tessellation_{region_id}.parquet"
    )
    print("processed tesselations")

---- Processing region:  4 2025-02-19 10:48:08.183355
Processed enclosures
processed tesselations
---- Processing region:  5 2025-02-19 10:48:13.862069
Retrying tesselation with less buildings, potentially changing building data.
Dropping 2 buildings due to tesselation problems
Processed enclosures
processed tesselations
---- Processing region:  20 2025-02-19 10:49:59.439670
Retrying tesselation with less buildings, potentially changing building data.
Dropping 69 buildings due to tesselation problems
Processed enclosures
processed tesselations
---- Processing region:  356 2025-02-19 11:24:24.559845
Processed enclosures
processed tesselations
---- Processing region:  401 2025-02-19 11:24:32.765773
Processed enclosures
processed tesselations
---- Processing region:  646 2025-02-19 11:24:39.609571
Retrying tesselation with less buildings, potentially changing building data.
Dropping 1 buildings due to tesselation problems
Processed enclosures
processed tesselations
---- Processing region:

/home/krasen/morphometrics/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Retrying tesselation with less buildings, potentially changing building data.


/home/krasen/morphometrics/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Dropping 4 buildings due to tesselation problems
Processed enclosures
processed tesselations
---- Processing region:  12502 2025-02-19 12:30:38.107246
Processed enclosures
processed tesselations
---- Processing region:  12511 2025-02-19 12:30:47.067722
Retrying tesselation with less buildings, potentially changing building data.
Dropping 1 buildings due to tesselation problems
Processed enclosures
processed tesselations
---- Processing region:  12551 2025-02-19 12:31:16.274645
Processed enclosures
processed tesselations
---- Processing region:  12590 2025-02-19 12:31:25.752540
Processed enclosures
processed tesselations
---- Processing region:  12647 2025-02-19 12:31:37.072447
Processed enclosures
processed tesselations
---- Processing region:  12815 2025-02-19 12:31:47.661319
Processed enclosures
processed tesselations
---- Processing region:  13347 2025-02-19 12:31:54.322254
Processed enclosures
processed tesselations
---- Processing region:  13348 2025-02-19 12:32:05.686970
Retrying

...

Dropping 7 buildings due to tesselation problems
Processed enclosures
processed tesselations
---- Processing region:  104700 2025-02-19 15:53:13.680200
Processed enclosures
processed tesselations
---- Processing region:  104774 2025-02-19 15:53:23.943003
Processed enclosures
processed tesselations
---- Processing region:  104942 2025-02-19 15:53:32.793672
Retrying tesselation with less buildings, potentially changing building data.
Dropping 1 buildings due to tesselation problems
Processed enclosures
processed tesselations
---- Processing region:  105139 2025-02-19 15:54:09.629234
Processed enclosures
processed tesselations
---- Processing region:  105549 2025-02-19 15:54:16.325603
Processed enclosures
processed tesselations
---- Processing region:  105598 2025-02-19 15:54:30.685911
Retrying tesselation with less buildings, potentially changing building data.
Dropping 1 buildings due to tesselation problems
Processed enclosures
processed tesselations
---- Processing region:  105620 202

/home/krasen/morphometrics/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Processed enclosures
processed tesselations
---- Processing region:  108938 2025-02-19 16:12:37.717700
Processed enclosures
processed tesselations
---- Processing region:  109204 2025-02-19 16:12:51.755299
Processed enclosures
processed tesselations
---- Processing region:  109339 2025-02-19 16:13:00.109561
Retrying tesselation with less buildings, potentially changing building data.
Dropping 1 buildings due to tesselation problems
Processed enclosures
processed tesselations
---- Processing region:  109648 2025-02-19 16:13:29.385055
Processed enclosures
processed tesselations
---- Processing region:  109769 2025-02-19 16:13:34.006390
Processed enclosures
processed tesselations
---- Processing region:  109915 2025-02-19 16:13:55.743062
Processed enclosures
processed tesselations
---- Processing region:  110149 2025-02-19 16:14:03.205839
Processed enclosures
processed tesselations
---- Processing region:  110569 2025-02-19 16:14:20.803036
Processed enclosures
processed tesselations
---- 

/home/krasen/morphometrics/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Processed enclosures
processed tesselations
---- Processing region:  125931 2025-02-19 16:29:24.898472
Processed enclosures
processed tesselations
---- Processing region:  126020 2025-02-19 16:29:31.061101
Processed enclosures
processed tesselations
---- Processing region:  126123 2025-02-19 16:29:56.722610
Processed enclosures
processed tesselations
---- Processing region:  126342 2025-02-19 16:30:12.147060
Processed enclosures
processed tesselations
---- Processing region:  126817 2025-02-19 16:30:18.925557
Processed enclosures
processed tesselations
---- Processing region:  126919 2025-02-19 16:30:39.774621
Processed enclosures
processed tesselations
---- Processing region:  127339 2025-02-19 16:32:12.289871
Processed enclosures
processed tesselations
---- Processing region:  127965 2025-02-19 16:32:36.298691
Processed enclosures
processed tesselations
---- Processing region:  128383 2025-02-19 16:32:49.419483
Processed enclosures
processed tesselations
---- Processing region:  1291

In [9]:
1

1

## Process graphs

Run the graph generating pipeline for all regions.

In [5]:
from core.generate_ngraphs import process_region_graphs

In [6]:
for region_id, _ in region_hulls.iterrows():

    process_region_graphs(
        region_id,
        graph_dir,
        buildings_dir,
        streets_dir,
        enclosures_dir,
        tessellations_dir,
    )

Built tess graph knn=1
Built buildings graph knn=1
Built streets graph knn=1
Built enclosure graph knn=1
Built nodes graph knn=1


/home/krasen/morphometrics/.pixi/envs/default/lib/python3.12/site-packages/libpysal/weights/weights.py:1685: UserWarning: The weights matrix is not fully connected: 
 There are 5 disconnected components.
  w = W(neighbors, weights, ids, silence_warnings=silence_warnings)


Built tess graph knn=1
Built buildings graph knn=1
Built streets graph knn=1
Built enclosure graph knn=1


/home/krasen/morphometrics/.pixi/envs/default/lib/python3.12/site-packages/libpysal/weights/weights.py:1685: UserWarning: The weights matrix is not fully connected: 
 There are 98 disconnected components.
  w = W(neighbors, weights, ids, silence_warnings=silence_warnings)


Built nodes graph knn=1
Built tess graph knn=1
Built buildings graph knn=1
Built streets graph knn=1
Built enclosure graph knn=1


...

/home/krasen/morphometrics/.pixi/envs/default/lib/python3.12/site-packages/libpysal/weights/weights.py:1685: UserWarning: The weights matrix is not fully connected: 
 There are 14 disconnected components.
  w = W(neighbors, weights, ids, silence_warnings=silence_warnings)


Built tess graph knn=1
Built buildings graph knn=1
Built streets graph knn=1
Built enclosure graph knn=1
Built nodes graph knn=1


/home/krasen/morphometrics/.pixi/envs/default/lib/python3.12/site-packages/libpysal/weights/weights.py:1685: UserWarning: The weights matrix is not fully connected: 
 There are 10 disconnected components.
  w = W(neighbors, weights, ids, silence_warnings=silence_warnings)


Built tess graph knn=1
Built buildings graph knn=1
Built streets graph knn=1
Built enclosure graph knn=1
Built nodes graph knn=1


/home/krasen/morphometrics/.pixi/envs/default/lib/python3.12/site-packages/libpysal/weights/weights.py:1685: UserWarning: The weights matrix is not fully connected: 
 There are 10 disconnected components.
  w = W(neighbors, weights, ids, silence_warnings=silence_warnings)


## Process morphometrics

Run the morphometric character processing pipeline for all regions.

In [7]:
from core.generate_chars import process_single_region_chars, process_building_chars

In [ ]:
for region_id, _ in region_hulls.iterrows():

    process_single_region_chars(
        region_id,
        graph_dir,
        buildings_dir,
        streets_dir,
        enclosures_dir,
        tessellations_dir,
        chars_dir
    )

2025-02-19 20:57:56.485802 ----Processing ------ 4
Processing streets
Processing enclosures
Processing buildings
Processing tessellation
2025-02-19 20:58:14.990728 ----Processing ------ 5
Processing streets
Processing enclosures
Processing buildings


/home/krasen/morphometrics/manual_deps/momepy/momepy/functional/_shape.py:890: RuntimeWarning: invalid value encountered in arccos
  angles = np.arccos(cosine_angle)
/home/krasen/morphometrics/manual_deps/momepy/momepy/functional/_shape.py:576: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(90 - degrees[true_angles]))
/home/krasen/morphometrics/manual_deps/momepy/momepy/functional/_shape.py:890: RuntimeWarning: invalid value encountered in arccos
  angles = np.arccos(cosine_angle)
/home/krasen/morphometrics/manual_deps/momepy/momepy/functional/_shape.py:737: RuntimeWarning: Mean of empty slice
  return Series({"mean": np.nanmean(dists), "std": np.nanstd(dists)})
/home/krasen/morphometrics/.pixi/envs/default/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/krasen/morphometrics/manual_deps/momepy/momepy/functional/_shape.py:890: RuntimeW

Processing tessellation


## Merge data

Merge all the building, street, nodes, enclosures and ETCs morphometric.

In [6]:
from core.generate_merged_primary_chars import merge_into_primary
from core.utils import used_keys

In [7]:
for region_id, _ in region_hulls.iterrows():
    tessellation = gpd.read_parquet(chars_dir + f"tessellations_chars_{region_id}.parquet")
    buildings = gpd.read_parquet(chars_dir + f"buildings_chars_{region_id}.parquet")
    enclosures = gpd.read_parquet(chars_dir + f"enclosures_chars_{region_id}.parquet")
    streets = gpd.read_parquet(chars_dir + f"streets_chars_{region_id}.parquet")
    nodes = gpd.read_parquet(chars_dir + f"nodes_chars_{region_id}.parquet")
    
    
    merged = pd.merge(
        tessellation.drop(columns=["geometry"]),
        buildings.drop(columns=["nodeID", "geometry", 'nID']),
        right_index=True,
        left_index=True,
        how="left",
    )
    
    merged = merged.merge(
        enclosures.drop(columns="geometry"),
        right_on="eID",
        left_on="enclosure_index",
        how="left",
    )
    
    merged = merged.merge(streets.drop(columns="geometry"), on="nID", how="left")
    merged = merged.merge(nodes.drop(columns="geometry"), on="nodeID", how="left")
    
    merged = merged.drop(
        columns=[
            "nID",
            "eID",
            "nodeID",
            "mm_len",
            "cdsbool",
            "node_start",
            "node_end",
            # "x",
            # "y",
            "enclosure_index",
            # "id",
            # "osm_id",
            # "index",  ## maybe keep
        ]
    )
    merged = merged.set_index(tessellation.index)
    
    primary = merged[list(used_keys.keys())]
    primary.to_parquet(chars_dir + f'primary_chars_{region_id}.parquet')

## Generate spatial lag

In [5]:
from core.generate_context import spatially_weighted_partial_lag, parallel_higher_order_context
import numpy as np
from libpysal.graph import read_parquet
import shapely
import geopandas as gpd
import pandas as pd

In [6]:
spatial_lag = 3


kernel ='inverse' 
n_splits = 10
bandwidth_type = -1

In [7]:
graph_dir = '/data/uscuni-eurofab/processed_data/neigh_graphs/'
chars_dir = '/data/uscuni-eurofab/processed_data/chars/'

regions_datadir = "/data/uscuni-eurofab/"
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "ms_ce_region_hulls.parquet"
    )
region_hulls.shape

(474, 1)

Spatially weighted context lag

In [8]:
%%time

for region_id in region_hulls.index:
    
    print(region_id)    
    
    # setup data for the spatial lag calculations
    X_train = pd.read_parquet(chars_dir + f'primary_chars_{region_id}.parquet')
    graph = read_parquet(graph_dir + f"tessellation_graph_{region_id}.parquet")
    tessellation = gpd.read_parquet(
            tessellations_dir + f"tessellation_{region_id}.parquet"
    )
    centroids = shapely.get_coordinates(tessellation.representative_point())
    
    # calculate and store the lag
    lag = spatially_weighted_partial_lag(X_train, graph, centroids, kernel=kernel, k=spatial_lag, n_splits=n_splits, bandwidth=bandwidth_type)
    combined_data = X_train.join(lag, how='inner')
    combined_data.to_parquet(f'{chars_dir}lag_chars_{region_id}_{kernel}_{spatial_lag}_{bandwidth_type}.parquet')

4
5
20
356
401
646
687
810
1014
1049
1054
1117
1321
1476
1598
1642
1827
1940
2110
2112
2268
2304
2485
2779
2846
2885
2997
3086
3147
3256
3335
3408
3415
3462
3631
3751
3758
3770
3888
3892
3907
4040
4066
4347
4356
4798
4833
5084
5388
5513
5832
6004
6037
6085
6090
6108
6574
6612
6738
6918
7098
7111
7125
7355
7607
7634
7674
7688
7823
7867
7962
8160
8213
8245
8272
8320
8360
8438
8854
9048
9147
9310
9353
9656
9678
9728
9754
9830
9871
9981
10078
10135
10196
10385
10446
10454
10574
10771
10846
10939
10957
10962
11055
11196
11282
11345
11633
11700
11785
11818
11855
11860
11944
12041
12214
12311
12363
12458
12493
12502
12511
12551
12590
12647
12815
13347
13348
13385
13475
13566
13633
13928
14064
14137
14242
14328
14404
14459
14496
14663
14759
14766
14946
15176
15241
15338
15351
15406
15416
15882
16414
16430
16439
17204
17373
17744
17780
17846
17892
17933
18116
18127
18230
19263
19442
19537
19811
19906
20034
20332
20490
20556
20796
20861
21115
21229
21365
21811
22026
22246
22417
22543
22552
23103

Equal spatial weights

In [8]:
%%time

for region_id in region_hulls.index:
    
    print(region_id)    
    
    # setup data for the spatial lag calculations
    X_train = pd.read_parquet(chars_dir + f'primary_chars_{region_id}.parquet')
    graph = read_parquet(graph_dir + f"tessellation_graph_{region_id}.parquet")

    # calculate and store the lag
    lag = parallel_higher_order_context(X_train, graph, k=spatial_lag, n_splits=n_splits, output_vals=3)
    lag.columns = np.concatenate([(c + "_15", c + "_median", c + "_85") for c in X_train.columns])
    combined_data = X_train.join(lag, how='inner')
    combined_data.to_parquet(f'{chars_dir}lag_chars_{region_id}_unweighted_{spatial_lag}.parquet')

19
24
33
478
754
817
1049
1485
1677
2415
2513
2707
2785
2790
2820
3228
3307
3313
3357
3540
3661
3762
3806
4271
4285
4640
4763
5175
5189
5320
5429
5874
6337
6351
6477
6858
6881
7113
7381
7411
7640
7693
7728
7921
7924
8014
8087
8147
8440
8659
8927
8960
9560
9840
9887
10197
10283
10600
10673
10764
10875
11024
11178
11550
11623
11640
12080
12222
12247
12347
12401
12546
12614
12649
12695
12736
13285
13496
13497
14086
14327
14383
14605
14836
15151
15308
15362
15415
15540
15560
15646
16446
16582
16687
17219
17720
17763
17808
17857
17874
17951
18006
18143
18215
19325
19474
20008
20063
20356
20573
20597
20811
21128
22022
22398
22633
22770
23227
23621
23941
24079
24141
24683
24737
25065
25497
25588
25814
25964
26146
26642
26773
27374
27700
27783
27997
28059
28060
28237
28601
28751
28795
28961
29387
30259
30571
30662
30938
31101
31696
31807
32671
32890
33094
33150
33415
33718
33769
33803
34553
34902
36043
36064
36227
36327
36396
36457
37246
37360
37371
37637
37789
37937
38374
38429
38584
38606
38